# Image Classification - MNIST

- dataset has 60,000 training images and 10,000 test images
- each image grayscale of size 28 by 28 pixels

### Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### Data Pipeline

In [2]:
transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.1307,), (0.3081,))
])

### Load MNIST Dataset

In [3]:
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

### Create DataLoaders

In [4]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

### Create Neural Network

In [5]:
class MNISTClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    # image size is 28 by 28 pixel (2D array) - Linear expects Flat vector (1D) = 784
    # Flatten converts 2D array to Flat vector
    # For train_dataset batch_size = 64
    # [64, 1, 28, 28] 
    #   64 - Batch Size
    #    1 - grayscale
    #   28 - image pixel size
    #   28 - image pixel size
    # flatten converts to [64, 784]
    self.flatten = nn.Flatten()
    self.layers = nn.Sequential(
      nn.Linear(784, 128),
      nn.ReLU(),  # Activation function
      nn.Linear(128, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    x = self.layers(x)
    return x

### Device and Optimizer

In [6]:
# check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

#initialize model and move to device
model = MNISTClassifier().to(device)

# loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

using cpu


### Training

In [7]:
def train_epoch(model, train_loader, loss_function, optimizer, device):
  model.train()
  running_loss = 0.0
  correct = 0
  total = 0

  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = loss_function(output, target)
    loss.backward()
    optimizer.step()

    # track progress
    running_loss += loss.item()
    _, predicted = output.max(1)
    total += target.size(0)
    correct += predicted.eq(target).sum().item()

    # print every 100 batches
    if batch_idx % 100 == 0 and batch_idx > 0:
      avg_loss  = running_loss/100
      accuracy = 100. * correct / total
      print(f" [{batch_idx * 64}/{60000} ]"
          f" Loss: {avg_loss:.3f} | Accuracy: {accuracy:.1f}%")
      running_loss = 0.0

### Evaluation

In [8]:
def evaluate(model, test_loader, device):
  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():
    for inputs, targets in test_loader:
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = model(inputs)
      _, predicted = outputs.max(1)
      total += targets.size(0)
      correct += predicted.eq(targets).sum().item()

  return 100. * correct / total

### Training Loop

In [9]:
num_epochs = 10

for epoch in range(num_epochs):
  print(f"\nEpoch: {epoch+1}")
  train_epoch(model, train_loader, loss_function, optimizer, device)
  accuracy = evaluate(model, test_loader, device)

  print(f"Test Accuracy: {accuracy:.2f}%")


Epoch: 1
 [6400/60000 ] Loss: 0.665 | Accuracy: 80.3%
 [12800/60000 ] Loss: 0.323 | Accuracy: 85.5%
 [19200/60000 ] Loss: 0.272 | Accuracy: 87.8%
 [25600/60000 ] Loss: 0.238 | Accuracy: 89.0%
 [32000/60000 ] Loss: 0.219 | Accuracy: 89.9%
 [38400/60000 ] Loss: 0.184 | Accuracy: 90.7%
 [44800/60000 ] Loss: 0.157 | Accuracy: 91.3%
 [51200/60000 ] Loss: 0.156 | Accuracy: 91.9%
 [57600/60000 ] Loss: 0.149 | Accuracy: 92.2%
Test Accuracy: 95.48%

Epoch: 2
 [6400/60000 ] Loss: 0.128 | Accuracy: 96.4%
 [12800/60000 ] Loss: 0.124 | Accuracy: 96.4%
 [19200/60000 ] Loss: 0.119 | Accuracy: 96.5%
 [25600/60000 ] Loss: 0.106 | Accuracy: 96.6%
 [32000/60000 ] Loss: 0.109 | Accuracy: 96.6%
 [38400/60000 ] Loss: 0.117 | Accuracy: 96.6%
 [44800/60000 ] Loss: 0.097 | Accuracy: 96.7%
 [51200/60000 ] Loss: 0.101 | Accuracy: 96.7%
 [57600/60000 ] Loss: 0.087 | Accuracy: 96.7%
Test Accuracy: 96.50%

Epoch: 3
 [6400/60000 ] Loss: 0.072 | Accuracy: 97.9%
 [12800/60000 ] Loss: 0.073 | Accuracy: 97.8%
 [19200/6